In [ ]:
import requests

In [2]:
previous_api_url = 'https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/structures/search'
new_api_url = 'https://scanr.dataesr.ovh/api/scanr-organizations/_search'

In [3]:
user_input = 'polymeres interfaces'

## 1. Previous query

In [4]:
body = {'query': user_input,
 'lang': 'fr',
 'page': '0',
 'pageSize': '20',
 'searchFields': ['label', 'alias'],
 'sourceFields': ['label', 'alias', 'id', 'address.city', 'kind'],
 'filters': {'kind': {'type': 'MultiValueSearchFilter',
   'op': 'any',
   'values': ['Structure de recherche', 'Secteur public']}}}

requests.post(previous_api_url, json=body).json()

{'request': {'query': 'polymeres interfaces',
  'page': 0,
  'pageSize': 20,
  'lang': 'fr',
  'searchFields': ['label', 'alias'],
  'filters': {'kind': {'type': 'MultiValueSearchFilter',
    'op': 'any',
    'values': ['Structure de recherche', 'Secteur public']}},
  'sourceFields': ['label', 'alias', 'id', 'address.city', 'kind'],
  'aggregations': {}},
 'total': 6,
 'results': [{'highlights': [{'type': 'label.fr',
     'value': '<strong>Polymères</strong>, colloïdes, <strong>interfaces</strong>'},
    {'type': 'label.default',
     'value': '<strong>Polymères</strong>, colloïdes, <strong>interfaces</strong>'},
    {'type': 'alias',
     'value': '<strong>POLYMERES</strong>, COLLOIDES, <strong>INTERFACES</strong>'}],
   'value': {'id': '200012192G',
    'kind': ['Structure de recherche'],
    'label': {'default': 'Polymères, colloïdes, interfaces',
     'fr': 'Polymères, colloïdes, interfaces'},
    'address': [{'city': 'Le Mans', 'main': False}],
    'alias': ['UMR 6120',
     'PCI'

## 2. New query

In [5]:
# scanr web UI uses this list of fields to search in (the ^ gives a weight):
                        # ['id^5', 'acronym.*^5',
                        #   'externalIds.id^3',
                        #   'alias^3',
                        #   'label.*^3',
                        #   'description.*^2',
                        #   'address.city',
                        #   'address.address',
                        #   'address.postcode',
                        #   'publications.id^2',
                        #   'publications.title.default^2',
                        #   'publications.summary.default',
                        #   'publications.domains.label.*^2',
                        #   'publications.domains.code^2',
                        #   'projects.label.*^2',
                        #   'projects.acronym.*',
                        #   'projects.id^2',
                        #   'badges.label.*',
                        #   'badges.code',
                        #   'patents.id.keyword^2',
                        #   'patents.title.*^2',
                        #   'agreements.type^2',
                        #   'awards.label^2',
                        #   'awards.summary',
                        #   'web_content']

In [8]:
aggregations = {'byInstitutions': {'terms': {'field': 'institutions.label.keyword', 'size': 10}}}
# for facets, not mandatory

In [14]:
new_body = {
    'query': {
        'bool': {
            'must': [
                {
                    'query_string': {
                        'query': user_input,
                        "default_operator": "AND",
                        'fields': [ 'alias', 'label.*']

                    }
                }
            ],
            'filter': [
                {'term': {'isFrench': True}}, 
                {'terms': {'kind.keyword': ['Structure de recherche', 'Secteur public']}}
             ],
         },
 },
 'from': '0',
 'size': '20',
 '_source': ['id', 'alias', 'label', 'kind', 'address.city'],
 'aggs': aggregations
}


requests.post(new_api_url, json=new_body).json()

{'took': 49,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 15.4590225,
  'hits': [{'_index': 'scanr-organizations-20240404',
    '_id': 'VlwvqY4BqgRIMCR9LqJS',
    '_score': 15.4590225,
    '_source': {'id': '200012192G',
     'label': {'fr': 'Polymères, colloïdes, interfaces',
      'default': 'Polymères, colloïdes, interfaces'},
     'kind': ['Structure de recherche'],
     'address': [{'city': 'Le Mans'}],
     'alias': ['UMR 6120',
      'PCI',
      'POLYMERES, COLLOIDES, INTERFACES',
      'Polymères, colloïdes, interfaces',
      'UMR 6120',
      'UMR6120']}},
   {'_index': 'scanr-organizations-20240404',
    '_id': 'VF9pqY4BZJJbNSwJNc93',
    '_score': 14.472206,
    '_source': {'id': '200214910C',
     'label': {'fr': 'Matériaux polymères aux interfaces',
      'default': 'Matériaux polymères aux interfaces'},
     'kind': ['Structure de recherche'],
     'address': 